# Fraudulent Transaction Detection


In [ ]:
%%capture
!apt update
!pip install kaggle
!pip install pandasql
!pip install sqlalchemy==1.4.46

In [ ]:
# import packages
import json
import glob
import pandas as pd
import pandasql as ps
from pandasql import sqldf
import numpy as np
import datetime as dt
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
from google.colab import drive
from sklearn.model_selection import train_test_split
from collections import Counter
import seaborn as sns
from scipy import stats
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, accuracy_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
import plotly.express as px

# **Part 1:** Preprocessing


We are using a transaction dataset from Kaggle.com that includes information such as time, transaction type, amount, isFraud, etc.

Data source: https://www.kaggle.com/datasets/arunavakrchakraborty/financial-payment-services-fraud-data

## **1.1** Data Loading and Preprocessing

### **1.1.1** Read and Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

In [ ]:
!!kaggle datasets download -d arunavakrchakraborty/financial-payment-services-fraud-data

['Downloading financial-payment-services-fraud-data.zip to /content',
 '',
 '  0% 0.00/178M [00:00<?, ?B/s]',
 '  1% 1.00M/178M [00:00<02:02, 1.51MB/s]',
 '  1% 2.00M/178M [00:00<01:02, 2.96MB/s]',
 '  2% 4.00M/178M [00:00<00:29, 6.24MB/s]',
 '  5% 8.00M/178M [00:01<00:13, 13.1MB/s]',
 '  6% 10.0M/178M [00:01<00:13, 13.4MB/s]',
 '  7% 13.0M/178M [00:01<00:10, 16.8MB/s]',
 '  9% 16.0M/178M [00:01<00:09, 18.6MB/s]',
 ' 11% 19.0M/178M [00:01<00:09, 17.5MB/s]',
 ' 12% 21.0M/178M [00:01<00:12, 13.5MB/s]',
 ' 14% 25.0M/178M [00:02<00:08, 17.8MB/s]',
 ' 16% 28.0M/178M [00:02<00:08, 19.6MB/s]',
 ' 17% 31.0M/178M [00:02<00:08, 19.2MB/s]',
 ' 19% 34.0M/178M [00:02<00:07, 21.4MB/s]',
 ' 21% 37.0M/178M [00:02<00:07, 20.5MB/s]',
 ' 23% 40.0M/178M [00:02<00:06, 22.7MB/s]',
 ' 24% 43.0M/178M [00:02<00:06, 23.1MB/s]',
 ' 26% 46.0M/178M [00:02<00:05, 24.4MB/s]',
 ' 28% 49.0M/178M [00:03<00:05, 22.7MB/s]',
 ' 29% 52.0M/178M [00:03<00:06, 19.0MB/s]',
 ' 31% 55.0M/178M [00:03<00:06, 21.2MB/s]',
 ' 33% 58.

In [ ]:
!unzip /content/financial-payment-services-fraud-data.zip

Archive:  /content/financial-payment-services-fraud-data.zip
  inflating: Fraud.csv               


In [ ]:
df_transactions = pd.read_csv('Fraud.csv')
df_transactions

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0


### **1.1.2** Understanding Data



In [ ]:
df_transactions.info(null_counts=True)

<ipython-input-9-68ba694b4f97>:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  df_transactions.info(null_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   step            6362620 non-null  int64  
 1   type            6362620 non-null  object 
 2   amount          6362620 non-null  float64
 3   nameOrig        6362620 non-null  object 
 4   oldbalanceOrg   6362620 non-null  float64
 5   newbalanceOrig  6362620 non-null  float64
 6   nameDest        6362620 non-null  object 
 7   oldbalanceDest  6362620 non-null  float64
 8   newbalanceDest  6362620 non-null  float64
 9   isFraud         6362620 non-null  int64  
 10  isFlaggedFraud  6362620 non-null  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [ ]:
df_transactions.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.798619e+05,8.338831e+05,8.551137e+05,1.100702e+06,1.224996e+06,1.290820e-03,2.514687e-06
std,1.423320e+02,6.038582e+05,2.888243e+06,2.924049e+06,3.399180e+06,3.674129e+06,3.590480e-02,1.585775e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,1.338957e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.390000e+02,7.487194e+04,1.420800e+04,0.000000e+00,1.327057e+05,2.146614e+05,0.000000e+00,0.000000e+00
75%,3.350000e+02,2.087215e+05,1.073152e+05,1.442584e+05,9.430367e+05,1.111909e+06,0.000000e+00,0.000000e+00
max,7.430000e+02,9.244552e+07,5.958504e+07,4.958504e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00


## **1.2** EDA



### **1.2.1** Visualization

#### (a) Examining Class Imbalance

In [ ]:
# Calculate type counts
type_counts = df_transactions['isFraud'].value_counts()

# Create a DataFrame for Plotly
data = pd.DataFrame({
    'Labels': ['Not Fraud', 'Fraud'],
    'Counts': type_counts.values
})

# Create a pie chart using Plotly
fig = px.pie(data, values='Counts', names='Labels', title='Distribution of Fraudulent Transactions',
             labels={'Counts': 'Count'}, hole=0.3)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(width=600, height=500)
fig.show()


We can see that the isFraud variable is very imbalanced. This is very common in fraud detection datasets, since in the real world, non-fraudulent transactions are far more common than fraudulent transactions. We will explore methods of working around this imbalanced data to improve model accuracies.

#### (b) Examining Distribution of Transaction Types


In [ ]:
type_counts = df_transactions['type'].value_counts()

# Create a DataFrame for Plotly
data = pd.DataFrame({
    'Labels': type_counts.index,
    'Counts': type_counts.values
})

# Create a pie chart using Plotly
fig = px.pie(data, values='Counts', names='Labels', title='Distribution of Transaction Types',
             labels={'Counts': 'Count'}, hole=0.3)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(width=600, height=500)
fig.show()

#### (c) Distribution of Fraud Across Different Transaction Types

In [ ]:
query = '''
SELECT
  LOWER(REPLACE(t.type, '_', ' ')) AS type,
  CASE WHEN t.isFraud = 1 THEN 'Fraud' ELSE 'Not Fraud' END AS isFraud,
  COUNT(*) * 100.0 / t2.Total AS Percentage
FROM df_transactions AS t
  INNER JOIN (SELECT isFraud, COUNT(*) AS Total FROM df_transactions GROUP BY isFraud) AS t2
    ON t.isFraud = t2.isFraud
GROUP BY t.type, t.isFraud'''

plotly_df = ps.sqldf(query, locals())

In [ ]:
fig = px.bar(plotly_df, x='isFraud', y='Percentage', color='type',
             labels={'isFraud': 'Fraud Status', 'Percentage': 'Percentage (%)'},
             title='Type Distribution for Fraud and Non-Fraud Transactions', barmode='group')

fig.update_layout(xaxis={'categoryorder': 'total descending'})  # Sort bars by total descending
fig.update_layout(width=600, height=500)

fig.show()

The graph shows us that transfer and cash out are the only types of transactions that can be fraudulent.

#### (d) Distribution of Fraud by Hour of Day

In [ ]:
df_timeofday = df_transactions
df_timeofday['step'] = df_timeofday['step'] % 24
df_timeofday = df_timeofday[df_timeofday['isFraud'] == 1]

hourly_fraud = df_timeofday.groupby('step').size()

plotly_df = pd.DataFrame({'Hour': hourly_fraud.index, 'Fraudulent Transactions': hourly_fraud.values})

# Create an interactive bar chart using Plotly
fig = px.bar(plotly_df, x='Hour', y='Fraudulent Transactions',
             labels={'Hour': 'Hour of Day', 'Fraudulent Transactions': 'Number of Fraudulent Transactions'},
             title='Number of Fraudulent Transactions By Hour of Day')

fig.update_layout(xaxis={'title': 'Hour of Day', 'tickvals': list(range(0, 24))})  # Setting x-axis ticks
fig.update_layout(width=900, height=500)

fig.show()

We can see that the fraudulent transactions are evenly distributed across hours of the day.



#### (e) Fraud Across Transaction Amount Ranges

To be able to visualize fraud across amount ranges effectively, we need to get rid of outlier transactions with very high amounts.

In [ ]:
z_scores = stats.zscore(df_transactions['amount'])
df_transactions_cleaned = df_transactions[(abs(z_scores) < 3)]

isFraudCount = len(df_transactions_cleaned[df_transactions_cleaned['isFraud'] == 1])
isNotFraudCount = len(df_transactions_cleaned[df_transactions_cleaned['isFraud'] == 0])
print(isFraudCount)
print(isNotFraudCount)

6527
6311148


In [ ]:
#what we want to do is have percentage of fraud belonging to each bin compared to all frauds. ex) count(fraud in first bin)/count(frauds)
min_amount = df_transactions_cleaned['amount'].min()
max_amount = df_transactions_cleaned['amount'].max()

bins = np.linspace(min_amount, max_amount, 6)

bin_labels = [f'{int(bins[i])}-{int(bins[i+1])}' for i in range(len(bins)-1)]

df_transactions_cleaned['Bin_Category'] = pd.cut(df_transactions_cleaned['amount'], bins=bins, labels=bin_labels)

df_fraud_cleaned = df_transactions_cleaned[df_transactions_cleaned['isFraud'] == 1]
df_not_fraud_cleaned = df_transactions_cleaned[df_transactions_cleaned['isFraud'] == 0]

hist_buckets_fraud = df_fraud_cleaned.groupby("Bin_Category")['step'].count().reset_index()
hist_buckets_fraud['Fraud Percentage'] = hist_buckets_fraud['step']/isFraudCount
hist_buckets_not_fraud = df_not_fraud_cleaned.groupby("Bin_Category")['step'].count().reset_index()
hist_buckets_not_fraud['Not Fraud Percentage'] = hist_buckets_not_fraud['step']/isNotFraudCount
hist_buckets_fraud = hist_buckets_fraud.drop(columns = ['step'])
hist_buckets_not_fraud = hist_buckets_not_fraud.drop(columns = ['step'])

hist_buckets_merged = hist_buckets_fraud.merge(hist_buckets_not_fraud, left_on = "Bin_Category", right_on = "Bin_Category")
hist_buckets_merged = hist_buckets_merged.melt(id_vars='Bin_Category', var_name='category', value_name='percentage')


<ipython-input-17-293617131b74>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
fig = px.bar(hist_buckets_merged, x="Bin_Category", y="percentage", color="category",
             labels={"Bin_Category": "Amount Ranges", "percentage": "Percentage of Fraudulent Transactions"},
             title="Percentage of Transactions Across Amount Ranges",
             barmode='group', orientation='v')

fig.update_layout(xaxis=dict(tickmode='array', tickvals=list(range(len(bin_labels))), ticktext=bin_labels),
                  yaxis_title="Percentage (%)", width=900, height=500)

fig.show()

We can see that most of the fraudulent transactions come from the dollar range 0-398286.

### **1.2.2** Correlation of Feature Variables and Feature Engineering




In [ ]:
from sklearn.preprocessing import OneHotEncoder
import plotly.figure_factory as ff

df_transactions
encoder = OneHotEncoder(sparse_output=False)
type_reshaped = df_transactions['type'].values.reshape(-1, 1)
type_encoded = encoder.fit_transform(type_reshaped)
type_encoded_df = pd.DataFrame(type_encoded, columns=encoder.get_feature_names_out(['type']))
df_encoded = pd.concat([df_transactions, type_encoded_df], axis=1)
df_encoded.drop('type', axis=1, inplace=True)

df_encoded['destBalanceDiff'] = df_encoded['newbalanceDest'] - df_encoded['oldbalanceDest']
df_encoded['orgBalanceDiff'] = df_encoded['newbalanceOrig'] - df_encoded['oldbalanceOrg']

df_encoded.drop(['step', 'nameOrig', 'nameDest', 'newbalanceOrig', 'oldbalanceOrg', 'newbalanceDest', 'oldbalanceDest'], axis=1, inplace=True)

heatmap_matrix = df_encoded[['amount', 'isFraud', 'isFlaggedFraud', 'type_CASH_IN', 'type_CASH_OUT', 'type_DEBIT', 'type_PAYMENT', 'type_TRANSFER', 'destBalanceDiff', 'orgBalanceDiff']]
columns = ['amount', 'isFraud', 'isFlaggedFraud', 'type_CASH_IN', 'type_CASH_OUT', 'type_DEBIT', 'type_PAYMENT', 'type_TRANSFER', 'destBalanceDiff', 'orgBalanceDiff']

corr_matrix = heatmap_matrix.corr()

# Define column names
column_names = heatmap_matrix.columns.tolist()

# Create annotated heatmap using Plotly
fig = ff.create_annotated_heatmap(z=corr_matrix.values, x=column_names, y=column_names,
                                  annotation_text=corr_matrix.round(2).values,
                                  colorscale='RdBu', showscale=True)

fig.update_layout(title='Correlation Heatmap for Various Numeric Features',
                  xaxis_title='Features', yaxis_title='Features')
fig.update_layout(width=700, height=700)
fig.update_xaxes(side="bottom")
fig.show()

Most of the features are largely uncorrelated. We removed initial balance and final balance and combined that into a single value: difference. This reduces correlation. The remaining correlated variables are amount and balance difference. These will hopefully be taken care of during PCA.

In [ ]:
df_encoded

,amount,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,destBalanceDiff,orgBalanceDiff
0,9839.64,0,0,0.0,0.0,0.0,1.0,0.0,0.00,-9839.64
1,1864.28,0,0,0.0,0.0,0.0,1.0,0.0,0.00,-1864.28
2,181.00,1,0,0.0,0.0,0.0,0.0,1.0,0.00,-181.00
3,181.00,1,0,0.0,1.0,0.0,0.0,0.0,-21182.00,-181.00
4,11668.14,0,0,0.0,0.0,0.0,1.0,0.0,0.00,-11668.14
...,...,...,...,...,...,...,...,...,...,...
6362615,339682.13,1,0,0.0,1.0,0.0,0.0,0.0,339682.13,-339682.13
6362616,6311409.28,1,0,0.0,0.0,0.0,0.0,1.0,0.00,-6311409.28
6362617,6311409.28,1,0,0.0,1.0,0.0,0.0,0.0,6311409.27,-6311409.28
6362618,850002.52,1,0,0.0,0.0,0.0,0.0,1.0,0.00,-850002.52


# **Part 2:** Modeling

In [ ]:
def test_results(y_test, y_pred, amounts_test):
  cm = confusion_matrix(y_test, y_pred)
  results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred, 'Amount': amounts_test})

  TP = cm[1, 1]
  FP = cm[0, 1]
  TN = cm[0, 0]
  FN = cm[1, 0]

  precision_transactions = TP / (TP + FP) if (TP + FP) else 0
  recall_transactions = TP / (TP + FN) if (TP + FN) else 0
  f1_score_transactions = 2 * precision_transactions * recall_transactions / (precision_transactions + recall_transactions) if (precision_transactions + recall_transactions) else 0


  TPR = TP / (TP + FN)  # True Positive Rate
  FPR = FP / (FP + TN)  # False Positive Rate
  TNR = TN / (TN + FP)  # True Negative Rate
  FNR = FN / (TP + FN)

  # Dollar Amount Metrics
  true_pred = results_df[results_df['Predicted'] == results_df['Actual']]
  wrong_pred = results_df[results_df['Predicted'] != results_df['Actual']]
  true_pred = results_df[results_df['Predicted'] == results_df['Actual']]
  wrong_pred = results_df[results_df['Predicted'] != results_df['Actual']]

  accuracy_transactions = (TP + TN) / (TP + FP + TN + FN) if (TP + FP + TN + FN) else 0

  total_dollar_amount = sum(results_df['Amount'])

  dollars_correctly_blocked = sum(true_pred[true_pred['Actual'] == 1]['Amount'])
  dollars_correctly_passed = sum(true_pred[true_pred['Actual'] == 0]['Amount'])
  dollars_incorrectly_blocked = sum(wrong_pred[wrong_pred['Predicted'] == 1]['Amount'])
  dollars_incorrectly_passed = sum(wrong_pred[wrong_pred['Predicted'] == 0]['Amount'])

  total_dollars_fraudulent = sum(results_df[results_df['Actual'] == 1]['Amount'])
  total_dollars_legitimate = sum(results_df[results_df['Actual'] == 0]['Amount'])

  rate_dollars_correctly_blocked = dollars_correctly_blocked / total_dollars_fraudulent if total_dollars_fraudulent else 0
  rate_dollars_correctly_passed = dollars_correctly_passed / total_dollars_legitimate if total_dollars_legitimate else 0
  rate_dollars_incorrectly_blocked = dollars_incorrectly_blocked / total_dollars_legitimate if total_dollars_legitimate else 0
  rate_dollars_incorrectly_passed = dollars_incorrectly_passed / total_dollars_fraudulent if total_dollars_fraudulent else 0


  accuracy_dollars = (dollars_correctly_blocked + dollars_correctly_passed) / total_dollar_amount if total_dollar_amount else 0

  precision_dollars = dollars_correctly_blocked / (dollars_correctly_blocked + dollars_incorrectly_blocked) if (dollars_correctly_blocked + dollars_incorrectly_blocked) else 0
  recall_dollars = dollars_correctly_blocked / (dollars_correctly_blocked + dollars_incorrectly_passed) if (dollars_correctly_blocked + dollars_incorrectly_passed) else 0
  f1_score_dollars = 2 * precision_dollars * recall_dollars / (precision_dollars + recall_dollars) if (precision_dollars + recall_dollars) else 0

  # Print statements
  print(f"True Positive Rate: {TPR}")
  print(f"False Positive Rate: {FPR}")
  print(f"True Negative Rate: {TNR}")
  print(f"False Negative Rate: {FNR}")

  print(f"Dollars Correctly Blocked: {dollars_correctly_blocked}")
  print(f"Dollars Corgirectly Passed: {dollars_correctly_passed}")
  print(f"Dollars Incorrectly Blocked: {dollars_incorrectly_blocked}")
  print(f"Dollars Incorrectly Passed: {dollars_incorrectly_passed}")

  print(f"Rate of Dollars Correctly Blocked: {rate_dollars_correctly_blocked}")
  print(f"Rate of Dollars Correctly Passed: {rate_dollars_correctly_passed}")
  print(f"Rate of Dollars Incorrectly Blocked: {rate_dollars_incorrectly_blocked}")
  print(f"Rate of Dollars Incorrectly Passed: {rate_dollars_incorrectly_passed}")

  print(f"Accuracy (Transactions): {accuracy_transactions}")
  print(f"Accuracy (Dollars): {accuracy_dollars}")

  print(f"F1 Score (Transactions): {f1_score_transactions}")
  print(f"F1 Score (Dollars): {f1_score_dollars}")



## **2.1** Baseline Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
#we have to include the one hot encoded column for type in the next line as well
features = df_encoded.drop(columns = ['isFraud'])
target = df_encoded['isFraud']
amount = df_encoded['amount']

In [ ]:
seed = 42
X_train, X_test, y_train, y_test, amounts_train, amounts_test = train_test_split(
    features, target, amount, test_size=0.2, random_state=seed
)

In [ ]:
clf = LogisticRegression(penalty = None, multi_class='multinomial').fit(X_train, y_train)
y_train_pred = clf.predict(X_train)
y_pred = clf.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
log_acc = clf.score(X_test, y_test)
log_acc #this is obviously high due to the fact that there is so much less fraudulent data than non fraudulent

0.9980126111570391

Confusion Matrix (make EVERYTHING neater at end)

In [ ]:
print("Train Results: ", accuracy_score(y_train, y_train_pred))
test_results(y_test, y_pred, amounts_test)

Train Results:  0.9979959120613835
True Positive Rate: 0.0006172839506172839
False Positive Rate: 0.0007160257580430937
True Negative Rate: 0.9992839742419569
False Negative Rate: 0.9993827160493827
Dollars Correctly Blocked: 21729.0
Dollars Corgirectly Passed: 227150034577.1128
Dollars Incorrectly Blocked: 31055843.48
Dollars Incorrectly Passed: 2228183372.639996
Rate of Dollars Correctly Blocked: 9.751795283121422e-06
Rate of Dollars Correctly Passed: 0.9998632991706207
Rate of Dollars Incorrectly Blocked: 0.00013670082938022987
Rate of Dollars Incorrectly Passed: 0.9999902482047168
Accuracy (Transactions): 0.9980126111570391
Accuracy (Dollars): 0.9901519281902836
F1 Score (Transactions): 0.0007902015013828526
F1 Score (Dollars): 1.9235308842850817e-05


After running logistic regression on the imbalanced dataset, we can see that our accuracies are extremely high. However, this is misleading because of the dominance of the majority class. Essentially, one can randomly guess every transaction to be not fraud, and these guesses would be correct most of the time because there are so many more non-fraudulent transactions than fraudulent ones. The F1-score is a better metric to use when considering imbalanced datasets, and we can see that the F1-scores are extremely low. We will now look into methods of balancing/modifying the dataset such that we achieve higher F1-scores.

Oversampling + Undersampling With RandomOver/UnderSampler

---



In [ ]:
# features
X = X_train
# target
y = y_train

# Oversample
ros = RandomOverSampler(sampling_strategy=0.5, random_state=1)
X_oversample, y_oversample = ros.fit_resample(X, y)

# Undersample
rus = RandomUnderSampler(sampling_strategy=1.0, random_state=1)
X_balanced, y_balanced = rus.fit_resample(X_oversample, y_oversample)

We used RandomOverSampler to generate fraudulent data until the number of fraudulent transactions was half the number of non-fraudulent transactions. Then, we used RandomUnderSampler to randomly remove non-fraudulent transactions until the number of fraudulent transactions and non-fraudulent transactions were equal. However, there don't seem to be enough transactions available to train on after undersampling.

Assign Weights to Rows With Higher Transactional Value

---

In [ ]:
# Weight Assigning Function to Be Used in Models

def calculate_weight(transaction, alpha=1.0):
  weight = transaction ** alpha
  return weight

weights = [calculate_weight(sample, 2.5) for sample in X_balanced['amount'].values]

Rather than the number of fraudulent transactions we catch, companies care more about the amount of money we are able to detect (companies wouldn’t be too happy if we could catch 100 \$10 frauds but miss one that transferred $1,000,000). Therefore, another method we will attempt is assigning weights to transactions with higher transactional value so that our models will focus more on accurately identifying high-value fraudulent transactions. To accomplish this, we gave more weight to data with higher transaction amounts by using the parameter sample_weights. We had to experiment with different parameters for alpha, the degree at which we emphasize transaction amounts.

PCA

---

In [ ]:
target_balanced = y_balanced
amount_balanced = X_balanced['amount']

#we will PCA transform X_balanced here

scaler = StandardScaler()
X_balanced_scaled = scaler.fit_transform(X_balanced)
X_test_scaled= scaler.transform(X_test)

pca = PCA()
pca.fit(X_balanced_scaled)
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)



In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# Add trace for cumulative variance
fig.add_trace(go.Scatter(x=np.arange(1, len(cumulative_variance) + 1), y=cumulative_variance,
                         mode='lines+markers', marker=dict(color='blue'), line=dict(shape='linear')))

# Update layout and axis labels
fig.update_layout(title='Explained Variance vs. Number of Components',
                  xaxis_title='Number of Components', yaxis_title='Cumulative Explained Variance')
fig.update_layout(width=900, height=500)

fig.show()

The 80% variance threshold is reached at 5 components, so we will do PCA with 5.

In [ ]:
#80 percent variance threshold
pca = PCA(n_components=5)
features_balanced_pca = pca.fit_transform(X_balanced_scaled)
test_features_pca = pca.transform(X_test_scaled)

Logistic Regression + Hyperparameter tuning

---

We decided to continue with logistic regression because it is a relatively simple classification model. We will then continue onto more complex models and compare them to logistic regression.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
lg_balanced = LogisticRegression(multi_class = 'multinomial',solver = 'saga')

param_grid = {
  'penalty': ['l1', 'l2', 'none'],  # Regularization types
}

grid_search = GridSearchCV(estimator=lg_balanced, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

grid_search.fit(features_balanced_pca, y_balanced, sample_weight = weights)

print("Best parameters found: ", grid_search.best_params_)

best_lg_balanced = grid_search.best_estimator_

y_train_pred = best_lg_balanced.predict(features_balanced_pca)
y_pred_balanced = best_lg_balanced.predict(test_features_pca)

# Evaluate the model
log_acc_balanced = best_lg_balanced.score(test_features_pca, y_test)
print("Logistic Regression Accuracy (Balanced):", log_acc_balanced)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



Best parameters found:  {'penalty': 'l2'}
Logistic Regression Accuracy (Balanced): 0.9848623680182064


In [ ]:
test_results(y_test, y_pred_balanced, amounts_test)

True Positive Rate: 0.6820987654320988
False Positive Rate: 0.014751704298672442
True Negative Rate: 0.9852482957013275
False Negative Rate: 0.31790123456790126
Dollars Correctly Blocked: 2190524095.8099995
Dollars Corgirectly Passed: 221414397712.1939
Dollars Incorrectly Blocked: 5766692708.399996
Dollars Incorrectly Passed: 37681005.830000006
Rate of Dollars Correctly Blocked: 0.9830890765835413
Rate of Dollars Correctly Passed: 0.9746163173276328
Rate of Dollars Incorrectly Blocked: 0.02538368267237298
Rate of Dollars Incorrectly Passed: 0.016910923416460253
Accuracy (Transactions): 0.9848623680182064
Accuracy (Dollars): 0.9746986114881916
F1 Score (Transactions): 0.1029199459786709
F1 Score (Dollars): 0.4301292800746669


Random Forest + Hyperparameter tuning

---

We are using Random Forest because it is a classification model that is generally reliable and robust to overfitting. We score on F1 to try and minimize misclassification.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Define your RandomForestClassifier
rf_classifier = RandomForestClassifier(random_state=42)

# Define a grid of hyperparameters to tune
# Here you can add more parameters and values according to your requirements
param_grid = {
    'n_estimators': [20, 30],  # Number of trees in the forest
    'max_depth': [None, 20],    # Maximum depth of the tree
}

# Instantiate the GridSearchCV object
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='f1')

# Fit the grid search to the data
grid_search.fit(features_balanced_pca, y_balanced, sample_weight=weights)

# Best parameters found
print("Best parameters found: ", grid_search.best_params_)

# Train the model using the best parameters
best_rf = grid_search.best_estimator_

# Predictions
y_train_pred = best_rf.predict(features_balanced_pca)
y_pred_rf = best_rf.predict(test_features_pca)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


In [ ]:
print("Train accuracy: ", accuracy_score(y_balanced, y_train_pred))
test_results(y_test, y_pred_rf, amounts_test)

XGBoost + Hyperparameter tuning

---
We are using XGBoost because it is a complex model that generally has higher accuracy, sensitivity, specificity, and precision than linear regression.

In [ ]:
!pip install xgboost

In [ ]:
import xgboost as xgb

In [ ]:
dtrain = xgb.DMatrix(features_balanced_pca, label=y_balanced, weight=weights)
dtest = xgb.DMatrix(test_features_pca, y_test)

xgb_clf = xgb.XGBClassifier(objective='binary:logistic', n_jobs=4, eval_metric='auc')

param_grid = {
    'max_depth': [3, 4],
    'learning_rate': [0.2, 0.3],
    'n_estimators': [50, 100],
}
grid_search = GridSearchCV(estimator=xgb_clf, param_grid=param_grid, cv=3, scoring='f1', verbose=2)

grid_search.fit(features_balanced_pca, y_balanced, sample_weight=weights)

# Best parameters and best score
print("Best parameters found: ", grid_search.best_params_)
print("Best ROC AUC score: ", grid_search.best_score_)

In [ ]:
optimized_xgb = xgb.XGBClassifier(
    learning_rate= 0.3,
    max_depth=8,
    n_estimators=100,
    objective='binary:logistic',
    n_jobs=4,
    eval_metric='auc'
)

optimized_xgb.fit(features_balanced_pca, y_balanced, sample_weight=weights)
y_pred_test = optimized_xgb.predict(test_features_pca)


In [ ]:
print("Train accuracy: ", accuracy_score(y_balanced, y_train_pred))
test_results(y_test, y_pred_test, amounts_test)